<a href="https://colab.research.google.com/github/apoorva2810/CS6910/blob/main/Assignment-3/DL_Assignment_3_without_attention_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
from keras.models import Model
from keras.layers import Input, LSTM, Dense, GRU,SimpleRNN
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense, RNN,GRU
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense,BatchNormalization
from keras.layers import Flatten
import math

from keras.layers import Dropout
import random
import os
from google.colab import drive
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm
from PIL import Image
import pickle
from keras.models import Model
from keras.layers import Input
from tensorflow.keras import layers, models, Model, optimizers
import tensorflow.keras as K

from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data_path1="/content/gdrive/MyDrive/hi.translit.sampled.dev.tsv"
data_path2="/content/gdrive/MyDrive/hi.translit.sampled.test.tsv"

with open(data_path2, "r", encoding="utf-8") as f:
    test = f.read().split("\n")

data_path3="/content/gdrive/MyDrive/hi.translit.sampled.train.tsv"

with open(data_path3, "r", encoding="utf-8") as f:
    train = f.read().split("\n")

In [ ]:
len(train)

44205

In [ ]:
# batch_size = 64  # Batch size for training.
# latent_dim = 256  # Latent dimensionality of the encoding space.



In [ ]:
data_path=data_path3

# Vectorize the data.
input_texts = []
target_texts = []

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: len(lines) - 1]:
    target_text, input_text,_ = line.split("\t")

    ##Appending '\t' and '\n' for every word in target_texts
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)




In [ ]:
input_texts[-1]

'om'

In [ ]:
"""Preparing set for unique imput and target characters"""
input_characters = set()
target_characters = set()

for i in range( len(input_texts)):
  for char in input_texts[i]:
    input_characters.add(char)

  for char in target_texts[i]:
    target_characters.add(char)


#Convert set to list
input_characters = list(input_characters)
target_characters = list(target_characters)

In [ ]:
"""Preparing dictionary for input characters and target characters"""
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

##Finding the length of maximum input word and maximum target word

max_encoder_seq_length=-1
for i in input_texts:
  if len(i)>max_encoder_seq_length:
    max_encoder_seq_length=len(i)

max_decoder_seq_length=-1
for i in target_texts:
  if len(i)>max_decoder_seq_length:
    max_decoder_seq_length=len(i)

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

target_token_index[" "]=65
input_token_index[" "]=26

In [ ]:
max_encoder_seq_length, max_decoder_seq_length

(20, 21)

In [ ]:
#Train and validation split of training data
inputTrain, inputValTrain, targetTrain, targetValTest = train_test_split(input_texts, target_texts,
                                                    test_size = 0.1,
                                                    random_state = 1) 

In [ ]:
print(len(inputTrain), len(targetTrain))
print(len(inputValTrain), len(targetValTest))

39783 39783
4421 4421


In [ ]:
encoder_input_data = np.zeros(
    (len(inputTrain), max_encoder_seq_length, num_encoder_tokens+1), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(inputTrain), max_decoder_seq_length, num_decoder_tokens+1), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(inputTrain), max_decoder_seq_length, num_decoder_tokens+1), dtype="float32"
)

In [ ]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(39783, 20, 27)
(39783, 21, 66)
(39783, 21, 66)


In [ ]:
"""Creating one hot vector for encoder inputs, decoder inputs, and decoder targer"""

for i, (input_text, target_text) in enumerate(zip(inputTrain, targetTrain)):

    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
   

for i, (input_text, target_text) in enumerate(zip(inputTrain, targetTrain)):

    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [ ]:
#For inputValTrain

encoder_input_val = np.zeros(
    (len(inputValTrain), max_encoder_seq_length, num_encoder_tokens+1), dtype="float32"
)

for i, input_text in enumerate(inputValTrain):
    for t, char in enumerate(input_text):
        encoder_input_val[i, t, input_token_index[char]] = 1.0
    encoder_input_val[i, t + 1 :, input_token_index[" "]] = 1.0

In [ ]:
"""Function returns word level accuracy and predicted words"""

def  ValAcc2(x,y,num_stack,model_type,enc_model,dec_model,beam_size):
  
  correct_predict=0
  predicted_words=[]
  for i, input_text in enumerate(x):
    
    # for t, char in enumerate(input_text):
    #     onehotlist[i, t, input_token_index[char]] = 1.0
    # onehotlist[i, t + 1 :, input_token_index[" "]] = 1.0

    input_seq = x[i:i+1]
    if num_stack==2:
      decoded_word = decode_sequence2(input_seq,model_type,enc_model,dec_model,beam_size)
    else :
       decoded_word = decode_sequence1(input_seq,model_type,enc_model,dec_model,beam_size)

    
    #print(decoded_word)
    #print("---",y[i])
    predicted_words.append(decoded_word)

    if decoded_word==y[i].split('\t')[1].split('\n')[0]:
      correct_predict=correct_predict+1
  
  
  valid_acc=correct_predict/len(x)    
  return valid_acc, predicted_words




